In [ ]:
import pandas as pd
import numpy as np
from itertools import product

In [ ]:
# for 12/08/18

SG = pd.read_csv('predict_SG_del_1208.csv')
PG = pd.read_csv('predict_PG_del_1208.csv')
G = pd.read_csv('predict_G_del_1208.csv')
C = pd.read_csv('predict_C_del_1208.csv')
F = pd.read_csv('predict_F_del_1208.csv')
PF = pd.read_csv('predict_PF_del_1208.csv')
SF = pd.read_csv('predict_SF_del_1208.csv')
UTL = pd.read_csv('predict_UTL_del_1208.csv')
real = pd.read_csv('game_log_1208.csv')

In [ ]:
# select top 20%
SG_new = SG.loc[SG['score'] > SG.score.describe([0.8])['80%']]
PG_new = PG.loc[PG['score'] > PG.score.describe([0.8])['80%']] 
G_new = G.loc[G['score'] > G.score.describe([0.8])['80%']] 
C_new = C.loc[C['score'] > C.score.describe([0.8])['80%']] 
F_new = F.loc[F['score'] > F.score.describe([0.8])['80%']] 
PF_new = PF.loc[PF['score'] > PF.score.describe([0.8])['80%']] 
SF_new = SF.loc[SF['score'] > SF.score.describe([0.8])['80%']] 
UTL_new = UTL.loc[UTL['score'] > UTL.score.describe([0.8])['80%']] 

In [ ]:
# create df with all possible combinations
team = pd.DataFrame(list(product(SG_new['name_SG'], PG_new['name_PG'], G_new['name_G'], C_new['name_C'], 
             F_new['name_F'], PF_new['name_PF'], SF_new['name_SF'], UTL_new['name_UTL'])))
team.columns = ['name_SG','name_PG', 'name_G','name_C', 'name_F','name_PF','name_SF','name_UTL']
team = pd.merge(team, SG_new, on = 'name_SG', how = 'left')
team = pd.merge(team, PG_new, on = 'name_PG', how = 'left')
team = pd.merge(team, G_new, on = 'name_G', how = 'left')
team = pd.merge(team, C_new, on = 'name_C', how = 'left')
team = pd.merge(team, F_new, on = 'name_F', how = 'left')
team = pd.merge(team, PF_new, on = 'name_PF', how = 'left')
team = pd.merge(team, SF_new, on = 'name_SF', how = 'left')
team = pd.merge(team, UTL_new, on = 'name_UTL', how = 'left')

In [ ]:
# calculate score and salary for the teams
team['score_sum'] = team['score_SG'] +team['score_PG'] +team['score_G'] + team['score_C'] +team['score_F'] +team['score_PF'] +team['score_SF'] + team['score_UTL']
team['salary_sum'] = team['salary_SG'] +team['salary_PG'] +team['salary_G'] + team['salary_C'] +team['salary_F'] +team['salary_PF'] +team['salary_SF'] + team['salary_UTL'] 

team_subset = team[['name_SG', 'name_PG', 'name_G', 'name_C', 'name_F', 'name_PF',
       'name_SF', 'name_UTL','score_sum', 'salary_sum']]

# cut at $50,000 limit
team_subset = team_subset.sort_values('salary_sum', ascending=False)

final = team_subset.loc[team_subset['salary_sum'] <= 50000].sort_values('score_sum', ascending=False)

In [ ]:
# assign one player only to one position
final_1 = final[(final['name_F'] != final['name_SF']) &
           (final['name_SG'] != final['name_UTL']) &
           (final['name_SG'] != final['name_F']) &
           (final['name_SG'] != final['name_G']) &
           (final['name_G'] != final['name_F']) &
           (final['name_SG'] != final['name_SF']) &
           (final['name_SF'] != final['name_UTL']) &
           (final['name_G'] != final['name_UTL']) &
           (final['name_PG'] != final['name_UTL']) &
           (final['name_C'] != final['name_UTL']) &
           (final['name_F'] != final['name_UTL']) &
           (final['name_PF'] != final['name_UTL']) &
           (final['name_F'] != final['name_PF']) &
           (final['name_G'] != final['name_SF']) &
           (final['name_PG'] != final['name_G']) &
           (final['name_SG'] != final['name_PG']) &
           (final['name_C'] != final['name_PF']) &
           (final['name_C'] != final['name_F']) &
           (final['name_PF'] != final['name_SF'])   ]
          
# the best team with the highest score is selected
chosen = final_1.iloc[0, 0:8]

In [ ]:
# predicted scores of the selected players
predicted = pd.concat([SG, PG, G, C, F, PF, SF, UTL])
p = predicted[predicted["name"].isin(final_1.iloc[0, 0:8])].drop_duplicates('name')[['name','salary', 'score']]

# obtain real scores after the game is over
r = real[real["game_log_today.namePlayer"].isin(final_1.iloc[0, 0:8])][['game_log_today.namePlayer', 'game_log_today.score']]
r.columns = ['name', 'score']

# compare predicted scores with the real scores
res = pd.merge(p,r, how='left', on='name')
res.loc['Total',:]= res[['salary', 'score_predicted', 'score_real']].sum(axis=0)